In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/MyDrive/Uni/UniPD/BioData/project/biological_data_pfp'

/content/drive/MyDrive/Uni/UniPD/BioData/project/biological_data_pfp


In [117]:
import h5py
import pandas as pd
import numpy as np
import networkx
import hashlib
import obonet
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def readh5_to_dict(file_path):
  # Create an empty dictionary to store the data
  p_embeddings_data = {}

  # Open the HDF5 file
  with h5py.File(file_path, 'r') as p_embeddings:
    # Store the data in the dictionary
    for key in p_embeddings.keys():
      p_embeddings_data[key] = p_embeddings[key][...]

  return p_embeddings_data

In [16]:
def sample_protein_ids(file_path,percentage):

  # Read the IDs from the text file
  with open(file_path, 'r') as file:
    ids = [line.strip() for line in file]

  # Calculate the index to get the first 30% of IDs
  split_index = int(len(ids) * percentage)

  # Select the first 30% of IDs
  selected_ids = ids[:split_index]

  return selected_ids

In [17]:
def read_tsv(tsv_file_path):
  # Read the TSV file into a Pandas DataFrame
  df_train_set = pd.read_csv(tsv_file_path, sep='\t')

  # Display the DataFrame
  return df_train_set

In [18]:
def read_dat(file_path):
  column_names = ['Protein_ID', 'IPR_ID', 'description', 'domain','dc1','dc2']
  df = pd.read_csv(file_path, delimiter='\t',names=column_names)

  return df


In [19]:
def filter_train_data(df, selected_ids, category):
  filtered_df = df[df['Protein_ID'].isin(selected_ids)]
  filtered_df = filtered_df[filtered_df['aspect'] == category]

  return filtered_df

In [20]:
def encode_go_terms(train_df):
  one_hot_encoding = pd.get_dummies(train_df['GO_term'])

  # Concatenate the one-hot encoded columns with the original DataFrame
  df_encoded = pd.concat([train_df, one_hot_encoding], axis=1)
  df_encoded_grouped = df_encoded.groupby('Protein_ID').sum().reset_index()

  return df_encoded_grouped

In [21]:
def encode_ipr_domain(df_ipr):
    df_ipr = df_ipr.drop(columns=['IPR_ID', 'description','dc1','dc2'])
    one_hot_encoding = pd.get_dummies(df_ipr['domain'],sparse=True)

    # Concatenate the one-hot encoded columns with the original DataFrame
    df_encoded = pd.concat([df_ipr, one_hot_encoding], axis=1)
    df_encoded_grouped = df_encoded.groupby('Protein_ID').sum().reset_index()

    return df_encoded

In [22]:
def get_embeddings(df, embeddings_dict):
  df['embedding'] = df['Protein_ID'].map(embeddings_dict)

  return df

In [23]:
def get_ipr(df_ipr,df_train):
   isp_dict = df_ipr.set_index('Protein_ID')['domain'].to_dict()
   df_train['ipr'] = df_train['Protein_ID'].map(isp_dict)

   return df_train

In [24]:
def create_y(df):
  y = df.to_numpy()
  return y


In [25]:
def create_X(df,variables):
  X = np.array(df[variables])
  X = np.vstack(X)

  return X

In [26]:
def get_top_freq(column, freq):
    top_freq = []
    for col in column:
        if col not in freq:
            continue
        top_freq.append(col)
    return top_freq

In [121]:
def make_array_key(array, context):
    return (array.tostring(), array.shape, array.dtype, context)

In [122]:
def get_prot_emd_pair(embeddings):
    return {make_array_key(v): k for k, v in embeddings.items()}

In [14]:
p_embeddings_data = readh5_to_dict('./dataset/train/train_embeddings.h5')
selected_ids = sample_protein_ids('./dataset/train/train_ids.txt', 1.0)
df_train_set = read_tsv('./dataset/train/train_set.tsv')
df_ipr = read_dat('./dataset/train/train_protein2ipr.dat')
df_train_set_filter = filter_train_data(df_train_set, selected_ids,'cellular_component')

In [123]:
test_embeddings_data = readh5_to_dict('./dataset/test/test_embeddings.h5')
test_ids = sample_protein_ids('./dataset/test/test_ids.txt', 1.0)
emb_prot_dat = get_prot_emd_pair(test_embeddings_data)

/tmp/ipykernel_19482/3174157729.py:2: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return (array.tostring(), array.shape, array.dtype)


In [110]:
X_test = []
for id in test_ids:
    X_test.append(test_embeddings_data[id])
X_test = np.array(X_test)

In [124]:
#df_ipr_encoded = encode_ipr_domain(df_ipr)
#df_ipr_encoded.head()
len(emb_prot_dat)

997

In [28]:
df_encoded = encode_go_terms(df_train_set_filter)
df_encoded = get_embeddings(df_encoded, p_embeddings_data)
#df_encoded = get_ipr(df_ipr,df_encoded)
df_encoded.head()

,Protein_ID,aspect,GO_term,GO:0000118,GO:0000123,GO:0000124,GO:0000131,GO:0000137,GO:0000138,GO:0000139,...,GO:1905360,GO:1905368,GO:1905369,GO:1990023,GO:1990204,GO:1990234,GO:1990351,GO:1990752,GO:1990904,embedding
0,A0A021WW32,cellular_componentcellular_componentcellular_c...,GO:0005575GO:0110165GO:0000785GO:0032991GO:000...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.01643, -0.001583, 0.00389, 0.0734, 0.01243..."
1,A0A021WZA4,cellular_componentcellular_componentcellular_c...,GO:0005575GO:0110165GO:0071944GO:0005886GO:001...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.007904, 0.0877, -0.001715, 0.03766, 0.01788..."
2,A0A023GPJ3,cellular_componentcellular_componentcellular_c...,GO:0005575GO:0110165GO:0005622GO:0005829GO:000...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.01512, 0.01102, 0.0217, -0.02512, 0.0396, 0..."
3,A0A023GUT0,cellular_componentcellular_componentcellular_c...,GO:0005575GO:0110165GO:0005576GO:0005615,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.00414, -0.01288, 0.0716, 0.01605, -0.03983..."
4,A0A023IM54,cellular_componentcellular_componentcellular_c...,GO:0005575GO:0005737GO:0042175GO:0032991GO:000...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.01651, 0.02525, 0.04333, 0.01558, -0.01678..."


In [29]:
df_encoded.isna().sum().sum()

0

In [30]:
df_encoded.columns[3:-1][df_encoded.iloc[1,3:-1] == 1]

Index(['GO:0005575', 'GO:0005886', 'GO:0016020', 'GO:0071944', 'GO:0110165'], dtype='object')

In [31]:
freq_df = pd.read_csv('./dataset/train/cellular_component_freq.csv')[:300]

In [32]:
y_columns = df_encoded.iloc[:, 3:-1]
pred_columns = get_top_freq(y_columns.columns.tolist(), set(freq_df['id']))

y_columns = y_columns[pred_columns]
y = create_y(y_columns)
X = create_X(df_encoded,'embedding')

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [56]:
embedding_size = len(X_train[1]) #1024
num_classes = len(y_train[1]) #678

def f1_score(y_true, y_pred):
    precision = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) / (K.sum(K.round(K.clip(y_pred, 0, 1))) + K.epsilon())
    recall = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) / (K.sum(K.round(K.clip(y_true, 0, 1))) + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())


# Number of folds
n_splits = 10
histories = []

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Iterate over each fold
fold_no = 1
for train_index, test_index in kf.split(X):
    # Splitting the data into training and testing sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Build a neural network model
    model = keras.Sequential([
        layers.Input(shape=(embedding_size,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='sigmoid')  # Sigmoid for multi-label classification
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_score])

    # Fit the model
    print(f'Training for fold {fold_no} ...')
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    histories.append(history)
    
    # Here, you can evaluate the model on the test set, e.g., calculate metrics
    results = model.evaluate(X_test, y_test)
    print(f"Test results - Fold {fold_no}: {model.metrics_names[0]} of {results[0]}; {model.metrics_names[1]} of {results[1]*100}%")

    fold_no += 1

Training for fold 1 ...
Epoch 1/10
2381/2381 [==============================] - 6s 1ms/step - loss: 0.0872 - accuracy: 0.8932 - f1_score: 0.6182 - val_loss: 0.0728 - val_accuracy: 0.8935 - val_f1_score: 0.6459
Epoch 2/10
2381/2381 [==============================] - 3s 1ms/step - loss: 0.0709 - accuracy: 0.8957 - f1_score: 0.6582 - val_loss: 0.0694 - val_accuracy: 0.8935 - val_f1_score: 0.6637
Epoch 3/10
2381/2381 [==============================] - 3s 1ms/step - loss: 0.0681 - accuracy: 0.8957 - f1_score: 0.6689 - val_loss: 0.0676 - val_accuracy: 0.8935 - val_f1_score: 0.6718
Epoch 4/10
2381/2381 [==============================] - 3s 1ms/step - loss: 0.0662 - accuracy: 0.8957 - f1_score: 0.6770 - val_loss: 0.0663 - val_accuracy: 0.8935 - val_f1_score: 0.6826
Epoch 5/10
2381/2381 [==============================] - 3s 1ms/step - loss: 0.0647 - accuracy: 0.8957 - f1_score: 0.6837 - val_loss: 0.0656 - val_accuracy: 0.8935 - val_f1_score: 0.6881
Epoch 6/10
2381/2381 [========================

In [68]:
final_model = keras.Sequential([
    layers.Input(shape=(embedding_size,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='sigmoid')
])

final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_score])
final_model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
2645/2645 [==============================] - 4s 1ms/step - loss: 0.0857 - f1_score: 0.6224
Epoch 2/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0704 - f1_score: 0.6600
Epoch 3/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0677 - f1_score: 0.6705
Epoch 4/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0659 - f1_score: 0.6781
Epoch 5/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0645 - f1_score: 0.6842
Epoch 6/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0633 - f1_score: 0.6896
Epoch 7/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0623 - f1_score: 0.6944
Epoch 8/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0614 - f1_score: 0.6989
Epoch 9/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.0606 - f1_score: 0.7026
Epoch 10/10
2645/2645 [==============================] - 3s 1ms/step - loss: 0.059

In [65]:
#Evaluate the model on the test set
y_pred = final_model.predict(X_test)

32/32 [==============================] - 0s 605us/step


### Post Processing

In [38]:
graph_df = pd.read_csv('./dataset/train/main_pairing.csv')

In [39]:
%%time
graph = obonet.read_obo('./dataset/taxonomy/go-basic.obo')

CPU times: user 4.24 s, sys: 95.9 ms, total: 4.34 s
Wall time: 4.34 s


In [75]:
### Propagating the probability of the children to the parent, in this case if the parent has several children, we will take the max probabilty of the children
def post_processing(y_pred, pred_scolumns, graph_df):
    new_preds = []

    # Get parent relation
    parent_dict = {}
    for _, row in graph_df.iterrows():
        parent_dict.setdefault(row['child'], []).append(row['parent'])

    for pred in y_pred:
        ### Build prediction dict
        preds = {k: 0 for k in pred_columns}
        new_pred = [0 for i in range(len(pred))]
        for i in range(len(pred)):
            term = pred_columns[i]
            preds[term] = pred[i]

        ### Search the probabilty for the parent
        pool = set()
        for term, prob in preds.items():
            for parent, child, key in graph.in_edges(term, keys=True):
                if key not in {'is_a', 'part_of'} or parent not in preds:
                    continue

                probability = max(prob, preds[parent])
                preds[parent] = probability
                    
        ### Build the array for the new preds
        for term, prob in preds.items():
            idx = pred_columns.index(term)
            new_pred[idx] = prob
        new_preds.append(new_pred)
    return np.array(new_preds)

In [78]:
new_y_pred = post_processing(y_pred, pred_columns, graph_df)

In [43]:
# Convert probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        46
           1       0.67      0.03      0.05        77
           2       0.38      0.03      0.05       107
           3       1.00      0.01      0.02       117
           4       0.69      0.49      0.57        72
           5       1.00      0.03      0.05       115
           6       0.69      0.04      0.08       271
           7       0.75      0.03      0.05       109
           8       1.00      0.03      0.07       117
           9       0.00      0.00      0.00        58
          10       0.63      0.31      0.42        70
          11       0.82      0.65      0.73        55
          12       0.70      0.17      0.28       122
          13       0.65      0.19      0.29        79
          14       0.71      0.24      0.36        84
          15       0.00      0.00      0.00        42
          16       0.73      0.07      0.13       328
          17       0.00    

/home/satria/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
# Convert probabilities to binary predictions
new_y_pred_binary = (new_y_pred > 0.5).astype(int)
print(classification_report(y_test, new_y_pred_binary))

/home/satria/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.33      0.24      0.28        46
           1       0.01      1.00      0.01        77
           2       0.01      1.00      0.02       107
           3       0.03      0.91      0.06       117
           4       0.01      1.00      0.02        72
           5       0.01      0.93      0.02       115
           6       0.02      0.92      0.05       271
           7       0.38      0.06      0.10       109
           8       0.01      0.97      0.02       117
           9       0.01      1.00      0.01        58
          10       0.63      0.31      0.42        70
          11       0.00      1.00      0.01        55
          12       0.21      0.25      0.23       122
          13       0.01      1.00      0.01        79
          14       0.60      0.29      0.39        84
          15       0.04      0.14      0.07        42
          16       0.03      1.00      0.05       328
          17       0.00    

In [151]:
out = {'id': [], 'term': [], 'score': []}
for i in range(len(new_y_pred)):
    for j in range(len(new_y_pred[i])):
        out['id'].append(test_ids[i])
        out['term'].append(pred_columns[j])
        out['score'].append(new_y_pred[i][j])

out_df = pd.DataFrame(out).reset_index(drop=True)

In [152]:
out_df = out_df.groupby('id', group_keys=False)
out_df = out_df.apply(lambda x: x.sort_values(by='score', ascending=False))

,id,term,score
197999,O43865,GO:1990904,1.000000
197987,O43865,GO:1902493,1.000000
197842,O43865,GO:0030880,1.000000
197843,O43865,GO:0030964,1.000000
197846,O43865,GO:0031248,1.000000
...,...,...,...
197908,O43865,GO:0043657,0.000278
197872,O43865,GO:0033646,0.000263
197907,O43865,GO:0043656,0.000263
197940,O43865,GO:0071011,0.000166


In [153]:
out = {}
for el in out_df.to_dict(orient='records'):
    out.setdefault(el['id'], {})
    out[el['id']][el['term']] = el['score']

submission = ''
for id in test_ids:
    for term, score in out[id].items():
        submission += f'{id} {term} {score}\n'
        
with open('submission.tsv', 'w') as file:
    # Write the string to the file
    file.write(submission)

In [156]:
import cafaeval
from cafaeval.evaluation import cafa_eval, write_results
res = cafa_eval("./dataset/taxonomy/go-basic.obo", "./submission.txt", "./dataset/test/test_ids.txt")
write_results(*res)

ValueError: not enough values to unpack (expected 2, got 1)